In [9]:
from pymystem3 import Mystem
from pprint import pprint
import subprocess
import requests
import json
import gensim
import pickle
from gensim import corpora
import nltk
import re
from sklearn import feature_extraction
import warnings
import time
warnings.simplefilter('ignore')

import logging
logging.basicConfig(filename='LDA_Train.log',format='%(levelname)-8s [%(asctime)s] (%(module)s : %(funcName)s) %(message)s',level=logging.INFO)
logging.info("application started")
logging.info("")
logging.info("")

# будем отображать графики прямо в jupyter'e
%pylab inline

#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5
import pandas as pd
import seaborn as sns

sns.set(rc=rcParams)
sns.set_style("darkgrid", {
    "axes.facecolor": "#282828",
    "axes.edgecolor":"#282828",
    "grid.color":"#298F4A",
    "xtick.color":"1",
    "ytick.color":"1",
    "text.color":"1",
    "axes.labelcolor":"1",
    "figure.facecolor":"#282828"
})
sns.set_palette(sns.color_palette("BrBG", 7))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import queue
poolS=queue.Queue()

THREAD_COUNT=4
for i in range(THREAD_COUNT):
    m=Mystem(mystem_bin='./mystem',entire_input=False)
    print(m.lemmatize("тест"))
    poolS.put(m)
    
%env PYRO_SERIALIZERS_ACCEPTED=marshal,json,serpent,pickle
%env PYRO_SERIALIZER=pickle

Populating the interactive namespace from numpy and matplotlib
['тест']
['тест']
['тест']
['тест']
env: PYRO_SERIALIZERS_ACCEPTED=marshal,json,serpent,pickle
env: PYRO_SERIALIZER=pickle


In [10]:
import numpy as np
%timeit() a=100
a=np.random.randint(0,100,size=(100,100))
b=np.random.randint(0,100,size=(100,100))
c=a@b
c

SyntaxError: invalid syntax (<unknown>, line 1)

In [ ]:
def lemmatize(text):
    m=poolS.get()
    lem=m.lemmatize(text)
    poolS.put(m)
    return lem
stopwords ="я а е и ж м о на не ни об но он мне мои мож она они оно мной много многочисленное многочисленная многочисленные многочисленный мною мой мог могут можно может можхо мор моя моё мочь над нее оба нам нем нами ними мимо немного одной одного менее однажды однако меня нему меньше ней наверху него ниже мало надо один одиннадцать одиннадцатый назад наиболее недавно миллионов недалеко между низко меля нельзя нибудь непрерывно наконец никогда никуда нас наш нет нею неё них мира наша наше наши ничего начала нередко несколько обычно опять около мы ну нх от отовсюду особенно нужно очень отсюда в во вон вниз внизу вокруг вот восемнадцать восемнадцатый восемь восьмой вверх вам вами важное важная важные важный вдали везде ведь вас ваш ваша ваше ваши впрочем весь вдруг вы все второй всем всеми времени время всему всего всегда всех всею всю вся всё всюду г\tгод говорил говорит года году где да ее за из ли же им до по ими под иногда довольно именно долго позже более должно пожалуйста значит иметь больше пока ему имя пор пора потом потому после почему почти посреди ей два две двенадцать двенадцатый двадцать двадцатый двух его дел или без день занят занята занято заняты действительно давно девятнадцать девятнадцатый девять девятый даже алло жизнь далеко близко здесь дальше для лет зато даром первый перед затем зачем лишь десять десятый ею её их бы еще при был про процентов против просто бывает бывь если люди была были было будем будет будете будешь прекрасно буду будь будто будут ещё пятнадцать пятнадцатый друго другое другой другие другая других есть пять быть лучше пятый к ком конечно кому кого когда которой которого которая которые который которых кем каждое каждая каждые каждый кажется как какой какая кто кроме куда кругом с\tт у я та те уж со то том снова тому совсем того тогда тоже собой тобой собою тобою сначала только уметь тот тою хорошо хотеть хочешь хоть хотя свое свои твой своей своего своих свою твоя твоё раз уже сам там тем чем сама сами теми само рано\
самом самому самой самого семнадцать семнадцатый самим самими самих саму семь чему раньше сейчас чего сегодня себе тебе сеаой человек разве теперь себя тебя седьмой спасибо слишком так такое такой такие также такая сих тех чаще четвертый через часто шестой шестнадцать шестнадцатый шесть четыре четырнадцать четырнадцатый сколько сказал сказала сказать ту ты три эта эти что это чтоб этом этому этой этого чтобы этот стал туда этим этими рядом тринадцать тринадцатый этих третий тут эту суть чуть тысяч"
stopwords +=" about after ago all almost along also any anybody anywhere are arent around ask been before being between but came can cant come could couldnt did didnt does doesnt dont each either else even every everybody everyone find for from get going gone got had has have havent having her here hers him his how ill into isnt its ive just know less like make many may more most much must near never none nothing now off often once one only other our ours out over please rather really said see she should small some something sometime somewhere take than thank thanks that thats the their theirs them then there these they thing think this those though through thus too true two under until upon use very want was way well were what when where which who whom whose why will with within without would yes yet you your yours без более больше большой был была были было быть вам вас вдоль ведь весь видно вместо вне вниз внизу внутри вокруг восемь вот все всегда всего всей всех всё вся где говорил говорили говорим говорить говорят год давай давать давно даже далеко два девять день десять для долго достаточно другого другой другое другие его если есть еще ещё зачем здесь знать ибо изо или именно иметь иной ином каждый каждого каждому каждым как какой какое когда который кроме кто либо лишь между меня мне много мог могу может мои мой навсегда над надо назад нам нас наш него недавно нее неё некто нельзя несколько нет никто них ноль оба обо один однако около она они оно оный опять особенно ото отчего очень под пожалуйста после потому похоже почему почти при про прямо пусть пять равно ребята редко сам самая сами самим самой самому самый самым самому свой себя семь сказал сказали сказать сначала снова совсем спасибо сразу среди стал стала стали стать ста сто так также такие такой там твой тем теперь тогда того тоже той только том тот точно три тут тысяч тысяча тысячи тысячу уже хотя хоть час часто часу чего чей чем четыре что чтоб чтобы чья чье чьё чья шесть эта эти это этого этом этот"
stopwords +=" a б в г е ж з м т у я є і аж ви де до за зі ми на не ну нх ні по та ти то ту ті це цю ця ці чи ще що як їй їм їх її або але ало без був вам вас ваш вже все всю вся від він два дві для ким мож моя моє мої міг між мій над нам нас наш нею неї них ніж ній ось при про під пір раз рік сам сих сім так там теж тим тих той тою три тут хоч хто цей цим цих час щоб яка які адже буде буду будь була були було бути вами ваша ваше ваші весь вниз вона вони воно всею всім всіх втім геть далі двох день дуже зате його йому каже кого коли кому крім куди лише люди мало мати мене мені миру мною може нами наша наше наші ними ніби один поки пора рано року році сама саме саму самі свою своє свої себе собі став суть така таке такі твоя твоє твій тебе тими тобі того тоді тому туди хоча хіба цими цієї часу чого чому який яких якої якщо ім'я інша інше інші буває будеш більш вгору вміти внизу вісім давно даром добре довго друго дякую життя зараз знову какая кожен кожна кожне кожні краще ледве майже менше могти можна назад немає нижче нього однак п'ять перед поруч потім проти після років самим самих самій свого своєї своїх собою справ такий також тепер тисяч тобою треба трохи усюди усіма хочеш цього цьому часто через шість якого іноді інший інших багато будемо будете будуть більше всього всьому далеко десять досить другий дійсно завжди звідси зовсім кругом кілька людина можуть навіть навіщо нагорі небудь низько ніколи нікуди нічого обидва одного однієї п'ятий перший просто раніше раптом самими самого самому сказав скрізь сьомий третій тільки хотіти чотири чудово шостий близько важлива важливе важливі вдалині восьмий говорив дев'ять десятий зайнята зайнято зайняті занадто значить навколо нарешті нерідко повинно посеред початку пізніше сказала сказати скільки спасибі частіше важливий двадцять дев'ятий зазвичай зайнятий звичайно здається найбільш не можна недалеко особливо потрібно спочатку сьогодні численна численне численні відсотків двадцятий звідусіль мільйонів нещодавно прекрасно четвертий численний будь ласка дванадцять одинадцять сімнадцять тринадцять безперервно дванадцятий одинадцятий одного разу п'ятнадцять сімнадцятий тринадцятий шістнадцять вісімнадцять п'ятнадцятий чотирнадцять шістнадцятий вісімнадцятий дев'ятнадцять чотирнадцятий дев'ятнадцятий"
stopwords = set(stopwords.split(" "))
stopwords.update(nltk.corpus.stopwords.words('russian'))
stopwords.update(nltk.corpus.stopwords.words('english'))
stopwords=set(lemmatize(' '.join(stopwords)))
"stopwrds count: %s"%len(stopwords)
# list(stopwords)[:50]

In [ ]:
def preprocessText(text):
    m = Mystem(mystem_bin='./mystem',entire_input=False)
    text=text.lower()
    text=re.sub(r"&.{1,10}?;","&",text)
    text=re.sub(r"br","",text) #remove most common html tag
#     print("!!!!",text)
    text=re.sub(r"((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z0-9\&\.\/\?\:@\-_=#])*"," ",text)#remove url
    text=re.sub(r"[_ ]"," ",text)
    text=re.sub(r"[^А-Яа-яA-zA-z- ]","",text)
    text=re.sub(r" +"," ",text)
    text=' '.join(lemmatize(text))
    text=list(filter(lambda x:x not in stopwords,text.split(" ")))
    return (text)
def prepareCorpus(csvs,dictionary=None,preprocess=True,gzip=False):
    texts=[]
    
    if type(csvs)==type(str()):
        csvs=[csvs]        
    try:
        iter(csvs)
        logging.debug('iteration will probably work')
    except TypeError:
        csvs=[csvs]
    if preprocess:
        posts=pd.DataFrame(columns=["text"])

        logging.info('start  loading texts')
        import gzip

        for csv in csvs:
            posts=posts.append(pd.read_csv(csv,header=None,names=["text"]),ignore_index=True)
    #         logging.info("non nul rows: %i",posts.count())
        logging.info('texts loaded')

        logging.info('preprocessing texts') 
        from multiprocessing.dummy import Pool as ThreadPool
        with ThreadPool(THREAD_COUNT) as pool:
            texts=pool.map(lambda x: preprocessText(str(x)[2:-2]),posts.values)
            logging.info(texts[:5])
            logging.info(posts.values[:5])
            logging.info('saving stammed texts')
            today=time.localtime()
            import gzip
            pickle.dump(texts,gzip.open('texts{}-{}-{}.bin.gzip'.format(today[2],today[1],today[0]),'wb'))
            logging.info('complete')
            logging.info('creating base fro corpus [[word for word in text] for text in texts]')
            texts = [[word for word in text] for text in texts]
            logging.info('preprocessing texts END')
    else:
        import gzip
        for csv in csvs:
            logging.info(csv)
            texts+=pickle.load(gzip.open('{}.bin.gzip'.format(csv),'rb'))    
#     print(texts[:10])
    if(dictionary==None):
        logging.info("create a Gensim dictionary from the texts")
        dictionary = corpora.Dictionary(texts)
#         dictionary = corpora.HashDictionary(texts)
        logging.info('dictionary length:  %i',len(dictionary))
    
        logging.info("remove extremes (similar to the min/max df step used when creating the tf-idf matrix)")
        dictionary.filter_extremes()
        logging.info('dictionary length after filtering:  %i',len(dictionary))
    
    logging.info("convert the dictionary to a bag of words corpus for reference")
    corpus = [dictionary.doc2bow(text) for text in texts]
    logging.info('corpus length:  %i',len(corpus))
    
    return (corpus,dictionary)
def trainModel(ldaModel,corpus):
    logging.info("learning model START")
    %time ldaModel.update(corpus=corpus)
    logging.info("learning model END!!!!!")
    
    import pickle
    logging.info("saving model")
    with open('LDA_themes:{}_passes:{}_terms:{}.pickle'.format(ldaModel.num_topics,ldaModel.num_updates,ldaModel.num_terms),'wb') as file:
        pickle.dump(ldaModel,file)
    logging.info("model saved")
    printModel(ldaModel)
    
def printModel(ldaModel):
    topics_matrix = ldaModel.show_topics(formatted=True, num_words=20,num_topics=-1)
    for i in topics_matrix:
        logging.info(i[0],i[1])
        logging.info("="*50)   
        
def trainClear(csvs):
    logging.info("prepare corpus START")
    corpus,dictionary=prepareCorpus(csvs)
    logging.info("prepare corpus END")
    
    import time
    today=time.localtime()
    pickle.dump(corpus,open('corpus{}-{}-{}.bin'.format(today[2],today[1],today[0]),'wb'))
    
    logging.info("creating model START")
    params={
        'num_topics':100,
#         'workers':2,
        'chunksize':4000,
        'passes':10,
        'id2word':dictionary,
    }
    ldaModel=gensim.models.LdaModel(distributed=True,**params) #sometimes slow, but faster than with learning (work in 1 threead)
    #(3 minutes on corpus length:  99877)
    logging.info("creating model END")
    
    trainModel(ldaModel,corpus)

def updateModel(csvs,model):
    import pickle
    ldaModel=pickle.load(open('./LDA_themes:100_passes:199754_terms:31793.pickle','rb'))
    dictionary=ldaModel.id2word
    
    logging.info("prepare corpus START")
    corpus,dictionary=prepareCorpus(csvs,dictionary=dictionary)
    logging.info("prepare corpus END")
    pickle.dump(corpus,open('corpus{}.bin'.format(str(csvs)),'wb'))
    
    trainModel(ldaModel,corpus)
    
preprocessText("смотрbrи — яbr <br>https://vk.com/supercards?mid=7485317&amp;fro&amp;m_id=10104402&amp;loc=s827i78592705 docker-compose — запускать и постгрес и нео теперь можно одной командой — docker-compose up")

In [12]:
csvs=[
    './csv/big-0.csv',
    './csv/big-1.csv',
    './csv/big-2.csv',
    './csv/big-3.csv',
    './csv/big-4.csv',
    './csv/big-5.csv',
    './csv/big-6.csv',
    './csv/big-7.csv',
    './csv/big-8.csv',
    './csv/big-9.csv',
    './csv/big-10.csv',
    './csv/big-11.csv',
    './csv/big-12.csv',
    './csv/big-13.csv',
    './csv/big-14.csv',
    './csv/big-15.csv',
    './csv/big-16.csv',
    './csv/big-17.csv',
    './csv/big-18.csv',
    './csv/big-19.csv',
]

In [13]:
import gzip
dictionary = corpora.Dictionary()
##TODO extract module build dictionary
# dictionary.add_documents
corpus=[]
for csv in csvs:
    logging.info(csv)
    texts=pickle.load(gzip.open('{}.bin.gzip'.format(csv),'rb'))    
    logging.info(len(texts))
    logging.info("create a Gensim dictionary from the texts")
    dictionary.add_documents(texts)

    logging.info('dictionary length:  %i',len(dictionary))
    logging.info("remove extremes (similar to the min/max df step used when crнeating the tf-idf matrix)")
    dictionary.filter_extremes()
    logging.info('dictionary length after filtering:  %i',len(dictionary))
    
    pickle.dump(dictionary,open('{}_dict.bin'.format(csv),'wb'))

pickle.dump(dictionary,open('dictionary_10G.bin'.format(csv),'wb'))
#     logging.info("convert the dictionary to a bag of words corpus for reference")
#     courpus = [dictionary.doc2bow(text) for text in texts]
#     logging.info('corpus length:  %i',len(corpus))


KeyboardInterrupt: 

In [14]:
##TODO extract module build corpus
corpus=[]
dictionary = dictionary
for csv in csvs:
    logging.info(csv)
    texts=pickle.load(gzip.open('{}.bin.gzip'.format(csv),'rb'))    
    logging.info(len(texts))
    logging.info("convert the dictionary to a bag of words corpus for reference")
    corpus = [dictionary.doc2bow(text) for text in texts]
    logging.info('corpus length:  %i',len(corpus))
    pickle.dump(corpus,open('{}_corpus.bin'.format(csv),'wb'))
    

In [ ]:
pickle.dump(corpus,open('corpus.bin'.format(csv),'wb'))

In [ ]:
csvs=[
    './csv/big-0.csv',
    './csv/big-1.csv',
    './csv/big-2.csv',
    './csv/big-3.csv',
    './csv/big-4.csv',
    './csv/big-5.csv',
    './csv/big-6.csv',
    './csv/big-7.csv',
    './csv/big-8.csv',
    './csv/big-9.csv',
    './csv/big-10.csv',
    './csv/big-11.csv',
    './csv/big-12.csv',
    './csv/big-13.csv',
    './csv/big-14.csv',
    './csv/big-15.csv',
    './csv/big-16.csv',
    './csv/big-17.csv',
    './csv/big-18.csv',
    './csv/big-19.csv',
]
logging.info("prepare corpus START")
##TODO extract module preprocess
for csv in csvs:
    logging.info('start %s'%csv)
    logging.info('loading ')
    posts=pd.read_csv(csv,header=None,names=["text"])
    logging.info('load  %s lines'%len(posts))
    from multiprocessing.dummy import Pool as ThreadPool
    with ThreadPool(THREAD_COUNT) as pool:
        
        texts=pool.map(lambda x: preprocessText(str(x)[2:-2]),posts.values)
        logging.info(texts[:5])
        logging.info(posts.values[:5])
        logging.info('saving stammed texts ащк')
        today=time.localtime()
        import gzip
        pickle.dump(texts,gzip.open('{}.bin.gzip'.format(csv),'wb'))
        logging.info('complete %s'%csv)


corpus,dictionary=prepareCorpus(csvs,preprocess=False)
import gzip
pickle.dump((corpus,dictionary),gzip.open('corp_dict_pair10G.bin.gzip','wb'))

In [8]:
# corpus     = pickle.load(open('corpus.bin','rb'))
import pickle
dictionary = pickle.load(open('dictionary_10G.bin','rb'))

In [ ]:
import time
import pickle
logging.info("readData")
today=time.localtime()
(corpus,dictionary)=pickle.load(open('corpus{}-{}-{}.bin'.format(today[2],today[1],today[0]),'rb'))
logging.info("End readData")

In [ ]:
logging.info("creating model START")
logging.info('courpus Length:'+len(corpus))
for i in range(1,100):   
    logging.info("courrent step:"+i)
    params={
        'num_topics':100,
        'chunksize':10000,
        'passes':10,
        'id2word':dictionary,
        'corpus':corpus[i*10000:(i+1)*10000],
        'update_every':5,
        'distributed':True,
        'ns_conf':{'host':'25.52.115.42'}
    }
    # ldaModel=gensim.models.LdaModel(corpus=corpus,distributed=True,id2word=dictionary,chunksize=4000,num_topics=100,passes=10) #sometimes slow, but faster than with learning (work in 1 threead)
#     ldaModel = gensim.models.LdaModel(
#     **params    
#     )
#     today=time.localtimerd':dictionary,
        'corpus':corpus[i*10000:(i+1)*10000],
        'update_every':5,
        '()
#     pickle.dump(ldaModel,open('model{}-{}-{}:{}.bin'.format(today[2],today[1],today[0],i),'wb'))
#(3 minutes on corpus length:  99877)
# logging.info("creating model END")
# printModel(ldaModel)
# trainModel(ldaModel,corpus)